Once you have run the first notebook for the registration, we can proceed to the ANTsPy registration. <br>
You should be running this notebook in your Linux subsystem, and have already uploaded the files your generated in the first notebook.

You can find the source code for **ANTsPy** [here](https://github.com/ANTsX/ANTsPy).

In [1]:
%matplotlib notebook

In [2]:
%load_ext autoreload

In [3]:
%autoreload

In [4]:
import numpy as np
import flammkuchen as fl
from pathlib import Path
import tifffile
import ants
import os
import pandas as pd
import getpass
import platform 

import ants_tools as utilities

from skimage.exposure import match_histograms
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed, interactive
from ipywidgets import widgets

from quickdisplay import *

## LOADING STACKS

In [5]:
master = "/work/data/landmarks/2p/ipn h2b"
path_list = list(Path(master).glob("*_f*"))
path_list


[PosixPath('/work/data/landmarks/2p/ipn h2b/240403_f0')]

In [6]:
fish_path = path_list[0]
print('Working on {}'.format(fish_path.name))

Working on 240403_f0


In [7]:
registration_dir = fish_path / "registration"

#Load stacks
ref_stack = fl.load(registration_dir / "ref_mapped.h5")
mov_stack = fl.load(registration_dir / "mov_mapped.h5" ) 


We will perform first some histogram scaling and matching.<br>
This can generally make the registration process a bit easier, but one should inspect the stack to make sure that the brighter regions are similar between the two stacks.

In [8]:
#Scale brightness
ref_stack = to_255_range(ref_stack, quantile_max=0.9999)
mov_stack = to_255_range(mov_stack, quantile_min=0.01, quantile_max=0.9999)

#Histogram matching
ref_stack = match_histograms(ref_stack, mov_stack)

In [9]:
%matplotlib widget
interact(plot_side_to_side,
        stack1=fixed(ref_stack),
        stack2=fixed(mov_stack),
        depth = widgets.IntSlider(min=0, max=100, step=5, continuous_update=False),
        dim = fixed(2),
        stack1_title=fixed('Ref stack'),
        stack2_title=fixed('Functional stack'))

interactive(children=(IntSlider(value=0, continuous_update=False, description='depth', step=5), Output()), _do…

<function quickdisplay.plot_side_to_side(stack1, stack2, depth, dim, stack1_title='Stack 1', stack2_title='Stack 2')>

## REGISTRATION

In [10]:
#Convert to AntsPy image format
ref_img = ants.from_numpy(ref_stack).clone("float")
mov_img = ants.from_numpy(mov_stack).clone("float")

Here we import an initial transformation matrix to help improve the result from the registration. If you don't have such matrix, go back to the Windows notebook, and run it to do some initial manual alignment. <br>
It is not essential, but definitely useful. If you want to skip this step, make sure to remove the `initial_transform` argument when calling the `ants.registration` function.

In [11]:
#Converting the initial transormation matrix (found with the manual annotations) to ANTs format
transform_mat = fl.load(registration_dir / "initial_transform_mapped.h5")
path_initial = str(registration_dir / "initial_transform_mapped.mat")
at = ants.create_ants_transform(transform_type='AffineTransform', precision='float', dimension=3,
                                matrix=transform_mat[:, :3], offset=transform_mat[:, 3])
ants.write_transform(at, path_initial)

### Affine registration

In [12]:
%%time
affine_tx = ants.registration(fixed=ref_img, 
                              moving=mov_img,
                              initial_transform=path_initial,
                              type_of_transform= 'TRSAA',
                              aff_metric='mattes',
                              aff_sampling=1,
                              total_sigma=0,
                              flow_sigma=0,
                              reg_iterations=[100, 50, 30],
                              verbose=False,
                              outprefix=str(registration_dir / 'affine_tx_')
                             )

CPU times: user 9min 55s, sys: 5.3 s, total: 10min
Wall time: 33.6 s


As an intermediate step, we can retrieve the morphed image and see how the affine transform performs by itself

In [13]:
#Retrieve output image
affine_stack = affine_tx['warpedmovout'].numpy()
fl.save(registration_dir / '{}_affine.h5'.format(fish_path.name), affine_stack)

In [14]:
interact(plot_overlay, 
         ref=fixed(ref_stack),
         mov=fixed(affine_stack),
         plane=widgets.IntSlider(min=0, max=ref_stack.shape[2], step=1, continuous_update=False),
         dim=fixed(2),
         stack1_title=fixed('Reference'),
         stack2_title=fixed('Affine movout')
)

interactive(children=(IntSlider(value=0, continuous_update=False, description='plane', max=65), Text(value='Re…

<function quickdisplay.plot_overlay(ref, mov, plane, dim, ref_title='Reference', mov_title='Warped movout')>

### Diffeomorphic registration

And now we can proceed with the diffeomorphic transformation. This time, when calling the morphing function, we will use the `SyNOnly` method, which performs only a deformation transformation, and we will specify as the initial transformation, the resulting affine derived in the previous step. <br>
Moreover, especially with these defomative tranformations, we want to play around a lot with the other parameters that we are passing to the registration function.

#### Load results from affine registration if needed

In [15]:
initial_affine_tx = str(next(registration_dir.glob('affine_tx*')))

In [16]:
%%time
warp_tx = ants.registration(fixed=ref_img,
                            moving=mov_img,
                            initial_transform=initial_affine_tx,
                            type_of_transform='SyNOnly',
                            syn_sampling=80, 
                            flow_sigma=10, 
                            reg_iterations=[15, 10, 5],
                            outprefix=str(registration_dir / 'warp_tx_'),
                            verbose=False
                           )

CPU times: user 4min 54s, sys: 18.8 s, total: 5min 13s
Wall time: 43 s


And we can finally retrieve the registered images and see how well they match the reference stack:

In [17]:
#Retrieve output image
warp_stack = warp_tx['warpedmovout'].numpy()
fl.save(registration_dir / '{}_warp.h5'.format(fish_path.name), warp_stack)

In [18]:
interact(plot_overlay,
         ref=fixed(ref_stack),
         mov=fixed(warp_stack),
         plane=widgets.IntSlider(min=0, max=ref_stack.shape[2]-1, step=1, continuous_update=False),
         dim = fixed(2),
         stack1_title=fixed('Reference'),
         stack2_title=fixed('Warped movout'))

interactive(children=(IntSlider(value=0, continuous_update=False, description='plane', max=64), Text(value='Re…

<function quickdisplay.plot_overlay(ref, mov, plane, dim, ref_title='Reference', mov_title='Warped movout')>

In [19]:
jacobian_det_mat_log = ants.create_jacobian_determinant_image(ref_img, warp_tx['fwdtransforms'][0], do_log=True)

In [20]:
interact(view_jacobian,
        ref=fixed(ref_stack),
        mov_affine=fixed(affine_stack),
        mov_warped=fixed(warp_stack),
        jacobian_mat=fixed(jacobian_det_mat_log),
        plane=widgets.IntSlider(min=0, max=ref_stack.shape[2]-1, step=1, continuous_update=False),
        dim = fixed(2))

interactive(children=(IntSlider(value=0, continuous_update=False, description='plane', max=64), Output()), _do…

<function quickdisplay.view_jacobian(ref, mov_affine, mov_warped, jacobian_mat, plane, dim)>

## COORDINATE TRANSFORMATION

And finally, we can import the coordinates for our ROIs and apply those same transformations to them.

In [21]:
#Pointer strings to transforms
affine_path = next(registration_dir.glob('warp_tx*Affine*'))
warp_path = next(registration_dir.glob('warp_tx*InverseWarp*'))

#Prepare antspy transfomration function inputs
tx_list = list([str(affine_path), str(warp_path)])
inversion_list = [True, False]

#Load ROI coordinates
mov_coords = fl.load(registration_dir / "mov_roi_coords_mapped.h5")

#Create DataFrame with coordinates to transform
mov_coords_df = pd.DataFrame(mov_coords, columns=['x', 'y', 'z'])

#Transform ROI coordinates
mov_coords_trans = ants.apply_transforms_to_points(3, mov_coords_df, transformlist=tx_list, whichtoinvert=inversion_list).values

#Store
fl.save(fish_path / 'mov_roi_coords_transformed.h5', mov_coords_trans)

In [22]:
#A check to make sure all ROIs are morphed inside the reference brain
plt.figure()

plt.imshow(np.nanmean(ref_stack, 2), cmap='gray')
plt.scatter(mov_coords_trans[:, 1], mov_coords_trans[:, 0], c='red', alpha=.075)

<IPython.core.display.Javascript object>

### POOLED LOOP FOR BATCH MORPHING

In [23]:
path_list = list(Path('/mnt/c/Users/portugueslab/python/ot/batch1').glob('*_f*'))
path_list

[]

In [24]:
for fish_path in path_list:
    
    print('Working on {}'.format(fish_path.name))
    registration_dir = fish_path

    #Load stacks
    ref_stack = fl.load(registration_dir / "ref_mapped.h5")
    mov_stack = fl.load(registration_dir / "mov_mapped.h5" ) 
    
    #Scale brightness
    ref_stack = to_255_range(ref_stack, quantile_max=0.9999)
    mov_stack = to_255_range(mov_stack, quantile_min=0.01, quantile_max=0.9999)

    #Histogram matching
    ref_stack = match_histograms(ref_stack, mov_stack)

    #Convert to AntsPy image format
    ref_img = ants.from_numpy(ref_stack).clone("float")
    mov_img = ants.from_numpy(mov_stack).clone("float")
    
    #Converting the initial transormation matrix (found with the manual annotations) to ANTs format
    transform_mat = fl.load(registration_dir / "initial_transform_mapped.h5")
    path_initial = str(registration_dir / "initial_transform_mapped.mat")
    at = ants.create_ants_transform(transform_type='AffineTransform', precision='float', dimension=3,
                                    matrix=transform_mat[:, :3], offset=transform_mat[:, 3])
    ants.write_transform(at, path_initial)

    #Affine registration
    print('Doing affine registration')
    affine_tx = ants.registration(fixed=ref_img, 
                                  moving=mov_img,
                                  initial_transform=path_initial,
                                  type_of_transform= 'TRSAA',
                                  aff_metric='mattes',
                                  aff_sampling=1,
                                  total_sigma=0,
                                  flow_sigma=0,
                                  reg_iterations=[100, 50, 30],
                                  verbose=False,
                                  outprefix=str(registration_dir / 'affine_tx_')
                                 )
    
    #Retrieve output image
    affine_stack = affine_tx['warpedmovout'].numpy()
    fl.save(registration_dir / '{}_affine.h5'.format(fish_path.name), affine_stack)
    
    #Retrieve affine transform
    initial_affine_tx = str(next(registration_dir.glob('affine_tx*')))
    
    #Warping
    print('Warping')
    warp_tx = ants.registration(fixed=ref_img,
                                moving=mov_img,
                                initial_transform=initial_affine_tx,
                                type_of_transform='SyNOnly',
                                syn_sampling=80, 
                                flow_sigma=10, 
                                reg_iterations=[15, 10, 5],
                                outprefix=str(registration_dir / 'warp_tx_'),
                                verbose=False
                               )
    
    #Retrieve output image
    warp_stack = warp_tx['warpedmovout'].numpy()
    fl.save(registration_dir / '{}_warp.h5'.format(fish_path.name), warp_stack)
    
    print('Registering ROI coordinates')
    #Pointer strings to transforms
    affine_path = next(registration_dir.glob('warp_tx*Affine*'))
    warp_path = next(registration_dir.glob('warp_tx*InverseWarp*'))

    #Prepare antspy transfomration function inputs
    tx_list = list([str(affine_path), str(warp_path)])
    inversion_list = [True, False]

    #Load ROI coordinates
    mov_coords = fl.load(registration_dir / "mov_roi_coords_mapped.h5")

    #Create DataFrame with coordinates to transform
    mov_coords_df = pd.DataFrame(mov_coords, columns=['x', 'y', 'z'])

    #Transform ROI coordinates
    mov_coords_trans = ants.apply_transforms_to_points(3, mov_coords_df, transformlist=tx_list, whichtoinvert=inversion_list).values

    #Store
    fl.save(fish_path / 'mov_roi_coords_transformed.h5', mov_coords_trans)